In [1]:
# pip install git+https://github.com/openai/swarm.git
# pip install langchain langchain_community langchain_ollama
# pip install arxiv2text
# pip install arxiv
# pip install pandas

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
model = os.getenv('LLM_MODEL')

In [3]:
from openai import OpenAI
from swarm import Swarm, Agent

ollama_client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'  # required but unused
)
#
client= Swarm(client=ollama_client)

In [4]:
import arxiv
import pandas as pd

def get_url_topic(topic):
    # Prompt user for the topic to search
    print(topic)
    #topic = "ChunkRag"
    # Set up the search parameters
    search = arxiv.Search(
        query=topic,
        max_results=1,  # You can adjust this number as needed
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending,
    )

    # Prepare a list to store results
    all_data = []

    # Execute the search and collect results
    for result in search.results():
        #print(result)
        paper_info = {
            "Title": result.title,
            "Date": result.published.date(),
            "Id": result.entry_id,
            "Summary": result.summary,
            "URL": result.pdf_url,
        }
        all_data.append(paper_info)

  
    if all_data:
        results = "\n\n".join([f"Title:{d['Title']}\nDate:{d['Date']}\nURL:{d['URL']}\nSummary:{d['Summary']}"for d in all_data])
        #print(results)
    return results

In [5]:
print(get_url_topic("ChunkRag"))

ChunkRag


/tmp/ipykernel_6409/2044149504.py:20: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Title:ChunkRAG: Novel LLM-Chunk Filtering Method for RAG Systems
Date:2024-10-25
URL:http://arxiv.org/pdf/2410.19572v4
Summary:Retrieval-Augmented Generation (RAG) systems using large language models
(LLMs) often generate inaccurate responses due to the retrieval of irrelevant
or loosely related information. Existing methods, which operate at the document
level, fail to effectively filter out such content. We propose LLM-driven chunk
filtering, ChunkRAG, a framework that enhances RAG systems by evaluating and
filtering retrieved information at the chunk level. Our approach employs
semantic chunking to divide documents into coherent sections and utilizes
LLM-based relevance scoring to assess each chunk's alignment with the user's
query. By filtering out less pertinent chunks before the generation phase, we
significantly reduce hallucinations and improve factual accuracy. Experiments
show that our method outperforms existing RAG models, achieving higher accuracy
on tasks requiring precis

In [10]:
from typing import List, Dict
from langchain_ollama import ChatOllama
from arxiv2text import arxiv_to_text
from openai import OpenAI

ollama_client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'  # required but unused
)

class SummarizerAgent:
    def __init__(self):
        self.llm = ChatOllama(model="llama3.2:1b",
                              temperature=0.0,
                              num_predict=1000)
    
    def extract_content(self,url:str) -> str:
        # Replace with your specific arXiv PDF URL
        pdf_url = url
        extracted_text = arxiv_to_text(pdf_url)
        return extracted_text

    def summarize_paper(self, paper: Dict,content: str) -> str:
        """
        Summarize a single paper using Llama2
        """
        prompt = f"""
        Please provide a concise summary of the following research paper:
        Title: {paper['title']}
        Authors: {', '.join(paper['authors'])}
        Abstract: {paper['summary']}
        Content : {content}
        
        Generate a clear ,concise and informative summary in no more than 6-8 sentences.
        """
        
        return self.llm.predict(prompt)

    def summarize_papers(self, papers: List[Dict]) -> List[Dict]:
        """
        Summarize multiple papers
        """
        summarized_papers = []
        for paper in papers:
            summary = self.summarize_paper(paper)
            summarized_papers.append({
                'title': paper['title'],
                'summary': summary,
                'original_paper': paper
            })
        
        return summarized_papers

In [11]:
from langchain_ollama import ChatOllama
llm =ChatOllama(model="llama3.2:1b",
                              temperature=0.0,
                              num_predict=1000)
#
def extract_content(url):
    summ = SummarizerAgent()
    content = summ.extract_content(url)
    return content

In [12]:
content = extract_content("http://arxiv.org/pdf/2410.19572v3")

In [13]:
url_agent = Agent(name="Extract URL Assistant",
                  instruction="Get the arxiv search results for the given topic.",
                  functions=[get_url_topic],
                  model=model)

In [14]:
extract_url = Agent(name="URL Assistant",
                  instruction="Get the URL from the given content.",
                  model=model)

In [15]:
content_agent = Agent(name="Extract Summary Assistant",
                  instruction="""Generate a clear ,concise and informative summary of the arxiv paper.The Summary should include the authors of the paper , the date it was published and 
                  the concept behind the topic explained i the paper.""",
                  functions=[extract_content],
                  model=model)

In [17]:
summary_response = client.run(
    agent=content_agent,
    messages=[{"role": "user", "content": content }],# retrive paper content
)
print(summary_response.messages[-1]["content"])

### Summary Report: System Development and Research

#### **Introduction**
The report summarizes the ongoing research and development work concerning various natural language processing (NLP) tasks, particularly those involving retrieval-augmented generation methods. These methods have been instrumental in enhancing information retrieval performance by integrating external knowledge sources back into downstream AI models.

#### **Research Methodology**

1. **Literature Review**: 
   - Comprehensive reviews of existing literature on self-reflective retrieval augmentation (self-reflective rag), fine-grained document retrieval for fact-checking tasks, multi-hop reasoning using graph-based methodologies, and corrective retrieval augmentation.
   - Detailed analyses focused on critical aspects such as error propagation, computational efficiency, and scalability.

2. **Experimental Setup**: 
   - Utilized benchmarks like RERAC (Retrieval-Augmented Generation), MetaRAG (Multi-meta-rag), Cove6

In [26]:
import re
def run_arxiv_paper_summary_workflow(topic):
    #Step1 Get the arxiv search results
    paper_details_response = client.run(agent=url_agent,
           messages=[{"role":"user","content":f"Get me the details for {topic}"}])
    text = paper_details_response.messages[-1]['content']
    print(text)
    # Step 2 Extract the URL from the search results
    url_response = client.run(agent=extract_url,
    messages=[{"role":"user","content":f"Get me the URL from the content{text}"}])
    #
    text2 = url_response.messages[-1]['content']
    print(text2)
    # Regex pattern to find URLs
    url_pattern = r'\((https?://[^\s)]+)\)'

    # Find all unique URLs in the text
    urls = set(re.findall(url_pattern, text2))

    # Print the unique URLs
    print(f"urls :{urls}")
    print(list(urls))
    #extract content form the url
    content = extract_content(list(urls)[0])
    print(f"Content :{content}")
    #Step 2 Generate Summary
    summary_response = client.run(
        agent=content_agent,
        messages=[{"role": "user", "content": content }],
    )
    print(summary_response.messages[-1]["content"])
    return summary_response.messages[-1]["content"]

In [27]:
topic = "ChunkRag"
run_arxiv_paper_summary_workflow(topic)

ChunkRag


/tmp/ipykernel_6409/2044149504.py:20: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


ChunkRAG
Based on the information provided:

- **Title**: ChunkRAG: Novel LLM-Chunk Filtering Method for RAG Systems
- **Date**: 2024-10-25
- **URL**: [http://arxiv.org/pdf/2410.19572v4](http://arxiv.org/pdf/2410.19572v4)
- **Summary**:
  Retrieval-Augmented Generation (RAG) systems using large language models
  (LLMs) often generate inaccurate responses due to the retrieval of irrelevant or loosely related information.
  Existing methods, which operate at the document level, fail to effectively filter out such content.
  
  We propose LLM-driven chunk filtering, ChunkRAG, a framework that enhances RAG systems by evaluating and filtering retrieved
  information at the chunk level. Our approach employs semantic chunking to divide documents into coherent sections and utilizes
  LLM-based relevance scoring to assess each chunk's alignment with the user’s query.
  
  By filtering out less pertinent chunks before the generation phase, we significantly reduce hallucinations and improve factu

KeyboardInterrupt: 

In [20]:
paper_details_response = client.run(agent=url_agent,
        messages=[{"role":"user","content":f"Get me the details for {topic}"}])
text = paper_details_response.messages[-1]['content']
print(text)


ChunkRag


/tmp/ipykernel_6409/2044149504.py:20: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Here are the details for ChunkRAG:

- **Title:** ChunkRAG: Novel LLM-Chunk Filtering Method for RAG Systems
- **Date:** 2024-10-25
- **URL:** [http://arxiv.org/pdf/2410.19572v4](http://arxiv.org/pdf/2410.19572v4)
- **Summary:**
Retrieval-Augmented Generation (RAG) systems using large language models often generate inaccurate responses due to the retrieval of irrelevant or loosely related information. Existing methods, which operate at the document level, fail to effectively filter out such content.

ChunkRAG is a proposed method in this paper that enhances RAG systems by filtering retrieved information at the chunk level. The framework uses semantic chunking to divide documents into coherent sections and LLM-based relevance scoring to assess each chunk's alignment with the user's query. By filtering out less pertinent chunks before the generation phase, it reduces hallucinations and improves factual accuracy.

Experiments have shown that this method outperforms existing RAG models on t

In [21]:
# Step 2 Extract the URL from the search results
url_response = client.run(agent=extract_url,
messages=[{"role":"user","content":f"Get me the URL from the content{text}"}])
#
text2 = url_response.messages[-1]['content']
print(text2)

The URL for the content you provided is:

[http://arxiv.org/pdf/2410.19572v4](http://arxiv.org/pdf/2410.19572v4)


In [22]:
# Regex pattern to find URLs
url_pattern = r'\((https?://[^\s)]+)\)'

# Find all unique URLs in the text
urls = set(re.findall(url_pattern, text2))

In [23]:
urls

{'http://arxiv.org/pdf/2410.19572v4'}

In [24]:
print(f"urls :{urls}")
print(list(urls))

urls :{'http://arxiv.org/pdf/2410.19572v4'}
['http://arxiv.org/pdf/2410.19572v4']


In [25]:
content = extract_content(list(urls)[0])
print(f"Content :{content}")

Content :ChunkRAG: Novel LLM-Chunk Filtering Method for RAG Systems

Ishneet Sukhvinder Singh*

Ritvik Aggarwal*

Aslihan Akalin

Kevin Zhu

Ibrahim Allahverdiyev
Sean O’Brien

Muhammad Taha

Algoverse AI Research
asli@algoverse.us, kevin@algoverse.us, seobrien@ucsd.edu

4
2
0
2

v
o
N
9
1

]
L
C
.
s
c
[

4
v
2
7
5
9
1
.
0
1
4
2
:
v
i
X
r
a

Abstract

Retrieval-Augmented Generation (RAG) sys-
tems using large language models (LLMs) of-
ten generate inaccurate responses due to the
retrieval of irrelevant or loosely related infor-
mation. Existing methods, which operate at the
document level, fail to effectively filter out such
content. We propose LLM-driven chunk filter-
ing, ChunkRAG, a framework that enhances
RAG systems by evaluating and filtering re-
trieved information at the chunk level, where
a "chunk" represents a smaller, coherent sec-
tion of a document. Our approach employs
semantic chunking to divide documents into
coherent sections and utilizes LLM-based rel-
evance scoring